In [1]:
!pip install transformers

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00


In [3]:
import zipfile
from google.colab import drive
import shutil
from transformers import AutoTokenizer, AutoModelForMaskedLM
import os
from datasets import Dataset
from transformers import BertForMaskedLM, BertTokenizer
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
from google.colab import files

In [4]:
# Load BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [5]:
# test the model
input_text = "The capital of France is [MASK]."
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model(**inputs)
print(outputs)

MaskedLMOutput(loss=None, logits=tensor([[[ -6.4346,  -6.4063,  -6.4097,  ...,  -5.7691,  -5.6326,  -3.7883],
         [-14.0119, -14.7240, -14.2120,  ..., -11.6976, -10.7304, -12.7618],
         [ -9.6561, -10.3125,  -9.7459,  ...,  -8.7782,  -6.6036, -12.6596],
         ...,
         [ -3.7861,  -3.8572,  -3.5644,  ...,  -2.5593,  -3.1093,  -4.3820],
         [-11.6598, -11.4274, -11.9266,  ...,  -9.8772, -10.2103,  -4.7594],
         [-11.7267, -11.7509, -11.8040,  ..., -10.5943, -10.9407,  -7.5151]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [ ]:
file_path = "/content/drive/My Drive/text.zip" ##Remember Change this to relative path

if os.path.exists(file_path):
    print(f"File Found: {file_path}")
else:
    print("Unfound")


找到文件: /content/drive/My Drive/text.zip


In [ ]:
zip_path = "/content/drive/My Drive/text.zip" #如果后面用文件夹 可以把这里改一下 不用解压

text_data = ""

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    txt_files = [f for f in zip_ref.namelist() if f.endswith('.txt')]

    print("压缩包中的 .txt 文件（前 5 个）:", txt_files[:5])  # 只看前 5 个

    for txt_file in txt_files:
        with zip_ref.open(txt_file) as file:
            text_data += file.read().decode("utf-8", errors="ignore") + "\n\n"

print(f"成功读取 {len(txt_files)} 个 .txt 文件并合并")


压缩包中的 .txt 文件（前 5 个）: ['text/300_0.txt', 'text/300_1.txt', 'text/300_10.txt', 'text/300_11.txt', 'text/300_12.txt']
成功读取 8690 个 .txt 文件并合并


In [8]:
# Choose a BERT model (you can replace it with 'bert-base-uncased' or other BERT variants)
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Process text
inputs = tokenizer(
    text_data,               # Your merged text data
    truncation=True,         # Truncate text exceeding 512 tokens
    padding=True,            # Pad to the maximum length
    max_length=512,          # Maximum number of tokens
    return_tensors="pt"      # Return tensors in PyTorch format
)

print("Tokenized result example:", inputs["input_ids"][0][:20])  # Display the first 20 tokens

Tokenized 结果示例: tensor([ 101, 8529, 2026, 3008, 2024, 2013, 2182, 8529, 1045, 2066, 1996, 6695,
        1045, 2052, 2031, 2042, 2763, 2041, 1999, 1996])


In [9]:
# Choose a model (you can replace it with 'bert-large-uncased', but it trains slower)
model_name = "bert-base-uncased"

# Load Tokenizer and BERT for MLM task
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Split text (process by lines)
lines = text_data.split("\n")  # Your psychological counseling conversation data

# Construct a HuggingFace Dataset
dataset = Dataset.from_dict({"text": lines})

# Tokenizer processing
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/17381 [00:00<?, ? examples/s]

In [11]:
# Masking process for the MLM task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15  # 15% of tokens will be masked
)

# Training parameters
training_args = TrainingArguments(
    output_dir="./bert_mlm_psych",  # Output path
    per_device_train_batch_size=8,  # Batch size per device (adjust to control training speed)
    num_train_epochs=2,  # Train for 1 epoch first to check results (can be changed to 3-5 epochs)
    save_steps=500,  # Save model every 500 steps during training
    save_total_limit=2,  # Keep only the latest 2 checkpoints
    logging_dir="./logs",  # Training logs
    report_to="none",  # Disable Wandb and other logging
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# Start training
trainer.train()

# Save trained model
model.save_pretrained("./bert_mlm_psych_model")
tokenizer.save_pretrained("./bert_mlm_psych_model")
print("Training completed, model saved!")



Step,Training Loss
500,3.096300
1000,2.943000
1500,2.863500
2000,2.621900
2500,2.703700
3000,2.479500
3500,2.387500
4000,2.393400


训练完成，模型已保存！


In [16]:
# Specify local path for `.pth` file
pth_path = "./bert_mlm_psych_model.pth"

# Save model parameters
torch.save(model.state_dict(), pth_path)

print(f"✅ `.pth` file saved at {pth_path}")


✅ `.pth` 文件已保存到 ./bert_mlm_psych_model.pth
